In [1]:
%cd /home/jnainani_umass_edu/codellm/MechInterpCodeLLMs

/home/jnainani_umass_edu/codellm/MechInterpCodeLLMs


In [2]:
import transformer_lens
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
import os
import time
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from torchtyping import TensorType as TT
from typing import List, Union, Optional, Callable
from typing_extensions import Literal
from functools import partial
import copy
import itertools
import json

from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets

import argparse

# THIS IS A LOCAL (MODIFIED) VERSION OF TRANSFORMER_LENS - UNINSTALL PIP/CONDA VERSION BEFORE USE!
import transformer_lens
import transformer_lens.utils as utils
import transformer_lens.patching as patching
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import (
    HookedTransformer,
    HookedTransformerConfig,
    FactoredMatrix,
    ActivationCache,
)

import utils_patch.patch_helper_functions as helper

import os
from config import HF_TOKEN, HF_PATH

os.environ["HF_TOKEN"] = HF_TOKEN
os.environ["TRANSFORMERS_CACHE"] = HF_PATH
os.environ["HF_DATASETS_CACHE"] = HF_PATH
os.environ["HF_HOME"] = HF_PATH

num_gpus = torch.cuda.device_count()
device_id = 0
if num_gpus > 0:
    device = "cuda:0"
else:
    device = "cpu"

    
def check_gpu_memory(max_alloc=0.9):
    if not torch.cuda.is_available():
        return
    global device_id, device
    print("Primary device:", device)
    torch.cuda.empty_cache()
    max_alloc = 1 if max_alloc > 1 else max_alloc
    for gpu in range(num_gpus):
        memory_reserved = torch.cuda.memory_reserved(device=gpu)
        memory_allocated = torch.cuda.memory_allocated(device=gpu)
        total_memory = torch.cuda.get_device_properties(gpu).total_memory 
        print(f"GPU {gpu}: {total_memory / (1024**2):.2f} MB  Allocated: {memory_allocated / (1024**2):.2f} MB  Reserved: {memory_reserved / (1024**2):.2f} MB")
                
        # Check if the current GPU is getting too full, and if so we switch the primary device to the next GPU
        if memory_reserved > max_alloc * total_memory:
            if device_id < num_gpus - 1:
                device_id += 1
                device = f"cuda:{device_id}"
                print(f"Switching primary device to {device}")
            else:
                print("Cannot switch primary device, all GPUs are nearly full")

print("Number of GPUs:", num_gpus)
check_gpu_memory()

/home/jnainani_umass_edu/.conda/envs/finetuning/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Number of GPUs: 0


In [4]:
model = HookedTransformer.from_pretrained("CodeLlama-7b-hf") #, n_devices=num_gpus)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model CodeLlama-7b-hf into HookedTransformer


In [3]:
model = HookedTransformer.from_pretrained("meta-llama/Llama-2-7b-hf")

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Loaded pretrained model meta-llama/Llama-2-7b-hf into HookedTransformer


# Iterate over a good prompt

In [87]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('experiments/old_trials/finetuneMI/data/objects.csv')

def generate_prompts(num_prompts=40):
    # operators = ['+', '-', '*', '**', '%', '<', '>', '<=', '>=', '=='] #['+', '-', '*', '**', '%']
    functions = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K']
    prompts = []
    
    for _ in range(num_prompts):
        sampled_df = df.sample(n=4)
        selected_operators = list(sampled_df.object_name)  #random.sample(operators, 4) #
        selected_functions = random.sample(functions, 4)
        prompt = "Variables: "
        # prompt = "#For this set of variables, return the variable name containing the specified item."
        
        examples = []
        op_to_func_mapping = {}

        for op, func in zip(selected_operators, selected_functions):
            prompt += f" '{func}' = {op}. "
            # prompt += f"    return a {op} b\n"
            # prompt += f"def {func}(a, b):\n"
            # prompt += f"    return a {op} b\n"
            examples.append([func,op])
            op_to_func_mapping[op] = func
        
        for i in range(4):
            chosen_op = selected_operators[i] #random.sample(selected_operators, 1)[0]
            prompt2add = prompt+ f" The name of the variable that has {chosen_op} is '"
            # prompt2add = prompt+f"Question: What is the name of the variable containing '{chosen_op}'? Answer: "
            prompts.append({"prompt": prompt2add, "output": op_to_func_mapping[chosen_op]})
    
    print(prompts[0]['prompt'])
    print(prompts[0]['output'])
    with open("data/info_retrieval/instructed_trial6_llama2.json", "w") as f:
        json.dump(prompts, f, indent=4)

generate_prompts()

Variables:  'K' = gift.  'J' = seed.  'I' = map.  'H' = chemical.  The name of the variable that has gift is '
K


In [95]:
import json
import re

# Function to transform the prompt using regex
def transform_prompt(original_prompt):
    # Extract key-value pairs from the original prompt
    pairs = re.findall(r"(\w) = '([^']+)'", original_prompt)
    
    # Extract the target item from the original prompt
    target_item = re.search(r"value '([^']+)'", original_prompt).group(1)
    
    # Create a dictionary from the pairs for easier lookup
    pairs_dict = {key: value for key, value in pairs}
    
    # Create the new format for the prompt
    # new_prompt = " ".join([f"The {value} is in Box {key}." for key, value in pairs]) + f" Name of the Box which has {target_item} is "
    new_prompt = " ".join([f"'{key}' has the '{value}'." for key, value in pairs]) + f" The '{target_item}' is in '"
    
    return new_prompt

# Read the JSON file
with open('data/info_retrieval/instructed_trial3.json', 'r') as infile:
    data = json.load(infile)

# Process each prompt-output pair
new_data = []
for pair in data:
    original_prompt = pair['prompt']
    transformed_prompt = transform_prompt(original_prompt)
    new_pair = {"prompt": transformed_prompt, "output": pair['output']}
    new_data.append(new_pair)

# Write the transformed pairs to a new JSON file
with open('data/info_retrieval/instructed_trial5_NL.json', 'w') as outfile:
    json.dump(new_data, outfile, indent=4)

print("Transformation complete. Check the output.json file for results.")


Transformation complete. Check the output.json file for results.


# Load and Calculate the number of tokens and prompts

In [88]:
with open('data/info_retrieval/instructed_trial6_llama2.json', 'r') as f:
    data = json.load(f)

# Take the first few dictionaries (e.g., first 3)
subset = data #[9:]

# Initialize lists
prompts = []
answers = []

# Extract prompts and outputs
outputs = []
for ind, item in enumerate(subset):
    # if ind in [0, 1, 3, 4, 5, 6, 7, 9]:
    prompts.append(item["prompt"])
    outputs.append(item["output"])

In [89]:
prompts[0]

"Variables:  'K' = gift.  'J' = seed.  'I' = map.  'H' = chemical.  The name of the variable that has gift is '"

In [90]:
len(prompts)

160

In [91]:
(model.to_tokens(prompts[20])).shape

torch.Size([1, 43])

In [92]:
model.generate(prompts[10], max_new_tokens=1)

  0%|          | 0/1 [00:00<?, ?it/s]

"Variables:  'A' = engine.  'G' = ball.  'J' = book.  'F' = key.  The name of the variable that has book is 'G"

# Test accuracy

In [93]:
def evaluate_next_token_accuracy(prompts, answers, model):
    # total_examples = len(prompts)
    correct_predictions = 0
    for ind in range(len(prompts)):
        # print(prompts[ind])
        with torch.no_grad():
            clean_tokens = model.to_tokens(prompts[ind])
            logits = model(clean_tokens)[:, -1, :]
            model_answers = torch.argmax(logits, dim=-1)
            print("pred: ",model.to_string(model_answers))
            # print("ans: ",answers[ind])
            if model.to_string(model_answers).lower() == answers[ind].lower():
                # print("pred: ",model.to_string(model_answers))
                print("ans: ",answers[ind])
                correct_predictions+=1
    return correct_predictions / len(prompts)

evaluate_next_token_accuracy(prompts[:50], outputs[:50], model)

pred:  K
ans:  K
pred:  J
ans:  J
pred:  I
ans:  I
pred:  H
ans:  H
pred:  C
ans:  C
pred:  C
pred:  P
pred:  M
pred:  A
ans:  A
pred:  G
ans:  G
pred:  J
ans:  J
pred:  F
ans:  F
pred:  I
ans:  I
pred:  D
ans:  D
pred:  K
ans:  K
pred:  A
ans:  A
pred:  F
ans:  F
pred:  K
ans:  K
pred:  G
pred:  J
ans:  J
pred:  E
ans:  E
pred:  F
ans:  F
pred:  G
ans:  G
pred:  C
ans:  C
pred:  E
ans:  E
pred:  I
ans:  I
pred:  A
ans:  A
pred:  F
ans:  F
pred:  A
ans:  A
pred:  G
ans:  G
pred:  H
ans:  H
pred:  J
ans:  J
pred:  E
ans:  E
pred:  B
ans:  B
pred:  B
pred:  I
ans:  I
pred:  K
ans:  K
pred:  C
ans:  C
pred:  J
ans:  J
pred:  H
ans:  H
pred:  J
ans:  J
pred:  D
pred:  C
pred:  H
ans:  H
pred:  E
pred:  C
ans:  C
pred:  M
pred:  K
ans:  K
pred:  D
ans:  D
pred:  H
ans:  H


0.82

In [79]:
model.generate(prompts[0], max_new_tokens=5)

  0%|          | 0/5 [00:00<?, ?it/s]

'lyh = "-"\njnh = "+"\nthj = "*"\nbgd = ""\nthe name of the variable with value "*" is 4\n\\end{'

In [84]:
# for prompt in prompts: 
#     clean_tokens = model.to_tokens(prompt)
#     logits = model(clean_tokens)[:, -1, :]
#     model_answers = torch.argmax(logits, dim=-1)
#     print(model_answers)
#     print(model.to_string(model_answers))

tensor([796], device='cuda:0')
Z
tensor([612], device='cuda:0')
Y
tensor([29896], device='cuda:0')
1


In [99]:
with open('data/  prompt_dataset_zero_shot.json', 'r') as f:
    data = json.load(f)

# Take the first few dictionaries (e.g., first 3)
subset = data[9:]

# Initialize lists
prompts = []
answers = []

# Extract prompts and outputs
outputs = []
for ind, item in enumerate(subset):
    # if ind in [0, 1, 3, 4, 5, 6, 7, 9]:
    prompts.append(item["prompt"])
    outputs.append(item["output"])

X = "-"
Y = "+"
Z = "*"
W = "<"
the name of the variable with value "*" is 


In [113]:
def evaluate_next_token_accuracy(prompts, answers, model):
    # total_examples = len(prompts)
    correct_predictions = 0
    for ind in range(len(prompts)):
        # print(prompts[ind])
        with torch.no_grad():
            clean_tokens = model.to_tokens(prompts[ind])
            logits = model(clean_tokens)[:, -1, :]
            model_answers = torch.argmax(logits, dim=-1)
            print("pred: ",model.to_string(model_answers))
            if model.to_string(model_answers) == answers[ind]:
                # print("pred: ",model.to_string(model_answers))
                print("ans: ",answers[ind])
                correct_predictions+=1
    return correct_predictions / len(prompts)

# evaluate_next_token_accuracy(prompts, answers, model)

In [114]:
evaluate_next_token_accuracy(prompts, answers, model)

pred:  Z
ans:  Z
pred:  Y
ans:  Y
pred:  1


0.6666666666666666

In [ ]:

import random
import json

def generate_prompts(num_prompts=400):
    operators = ['+', '-', '*', '**', '%', '<', '>', '<=', '>=', '==']
    functions = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K']
    prompts = []
    
    for _ in range(num_prompts):
        selected_operators = random.sample(operators, 11)
        selected_functions = random.sample(functions, 11)
        
        prompt = "For this set of functions, return the function which uses the specified operator.\n\n"
        
        examples = []
        op_to_func_mapping = {}

        for op, func in zip(selected_operators, selected_functions):
            prompt += f"def {func}(a, b):\n"
            prompt += f"    return a {op} b\n"
            examples.append([func,op])
            op_to_func_mapping[op] = func
        

        
        prompt += f"\nExample: the name of the function that uses the '{examples[0][1]}' operator is "
        prompt += examples[0][0]
        prompt += f"\nExample: the name of the function that uses the '{examples[1][1]}' operator is "
        prompt += examples[1][0]

        selected_operators.remove(examples[0][1])
        selected_operators.remove(examples[1][1])

        chosen_op = random.sample(selected_operators, 1)[0]

        prompt += f"\nExample: the name of the function that uses the '{chosen_op}' operator is "

        
        prompts.append({"prompt": prompt, "output": op_to_func_mapping[chosen_op]})
    
    print(prompts[0]['prompt'])
    print(prompts[0]['output'])
    with open("prompts_dataset.json", "w") as f:
        json.dump(prompts, f, indent=4)

generate_prompts()

In [3]:
!cd /Users/zephyr/Documents/mechReasonCodeLLM/MechInterpCodeLLMs

In [4]:
!pwd

/Users/zephyr/Documents/mechReasonCodeLLM/MechInterpCodeLLMs/experiments/InfoRetrieval


In [12]:

import random
import json

def generate_prompts(num_prompts=40):
    operators = ['+', '-', '*', '**', '%']
    functions = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K']
    prompts = []
    
    for _ in range(num_prompts):

        selected_operators = random.sample(operators, 5)
        selected_functions = random.sample(functions, 5)
        prompt = ""
        # prompt = "For this set of variables, return the variable name containing the specified operator.\n"
        
        examples = []
        op_to_func_mapping = {}

        for op, func in zip(selected_operators, selected_functions):
            prompt += f"{func}= {op} \n"
            # prompt += f"    return a {op} b\n"
            examples.append([func,op])
            op_to_func_mapping[op] = func
        
        for i in range(5):
            chosen_op = selected_operators[i] #random.sample(selected_operators, 1)[0]
            prompt2add = prompt+ f"The name of the variable that contains the '{chosen_op}' operator is "
            prompts.append({"prompt": prompt2add, "output": op_to_func_mapping[chosen_op]})
    
    print(prompts[0]['prompt'])
    print(prompts[0]['output'])
    with open("../../data/info_retrieval/trial1.json", "w") as f:
        json.dump(prompts, f, indent=4)

generate_prompts()

C= * 
D= % 
F= ** 
J= + 
G= - 
The name of the variable that contains the '*' operator is 
C


In [ ]:
['document', 'pot', 'magnet', 'game', 'cross', 'map']

For this set of variables, return the variable name containing the specified operator.
D = %
B = -
K = *
F = **
I = +
The name of the variable that contains the '%' operator is 
D


D = %
B = -
K = *
F = **
I = +
O = 


In [16]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('../old_trials/finetuneMI/data/objects.csv')

# Randomly sample 5 rows from the DataFrame
sampled_df = df.sample(n=5)

# Display the sampled rows
print(list(sampled_df.object_name))

['plant', 'leaf', 'magnet', 'key', 'brain']


In [24]:

def generate_prompts(num_prompts=40):
    # operators = ['+', '-', '*', '**', '%']
    functions = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K']
    prompts = []
    
    for _ in range(num_prompts):
        sampled_df = df.sample(n=4)
        selected_operators = list(sampled_df.object_name) #random.sample(operators, 5)
        selected_functions = random.sample(functions, 4)
        prompt = ""
        # prompt = "For this set of variables, return the variable name containing the specified operator.\n"
        
        examples = []
        op_to_func_mapping = {}

        for op, func in zip(selected_operators, selected_functions):
            prompt += f"{func} = ['{op}'] \n"
            # prompt += f"    return a {op} b\n"
            examples.append([func,op])
            op_to_func_mapping[op] = func
        
        for i in range(4):
            chosen_op = selected_operators[i] #random.sample(selected_operators, 1)[0]
            prompt2add = prompt+ f"The name of the variable that contains '{chosen_op}' is "
            prompts.append({"prompt": prompt2add, "output": op_to_func_mapping[chosen_op]})
    
    print(prompts[0]['prompt'])
    print(prompts[0]['output'])
    with open("../../data/info_retrieval/trial4.json", "w") as f:
        json.dump(prompts, f, indent=4)

generate_prompts()

J = ['chemical'] 
G = ['ball'] 
D = ['fan'] 
E = ['brain'] 
The name of the variable that contains 'chemical' is 
J


J = ['chemical'] 
G = ['ball'] 
D = ['fan'] 
E = ['brain'] 

## Example from entity tracking paper:
move -> Move the chemical from J to E.
## List operation version
E.append(J)
J.pop()

## Example from entity tracking paper:
remove -> Remove the fan from D.
## List operation version
D.pop()

## Example from entity tracking paper:
put -> Put the car into G.
## List operation version
G.append('car')


## Example queries: 

1. The name of the variable that contains 'chemical' is
### before move 
-> J
### after move
-> E

1. The name of the variable that contains 'fan' is
### before remove 
-> J
### after remove
-> none


1. The name of the variable that contains 'car' is
### before put 
-> none
### after put
-> G
